In [ ]:
%run default-imports.ipynb
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

In [ ]:
cohorts = {
    'MIMIC' : { 'title' : "MIMIC-III", 'filename' : "./experiments/mimic_model_results_calibration.d" },
    'SINAI' : { 'title' : "Mt. Sinai", 'filename' : "./experiments/sinai_model_results.d" },
    'DHZB'  : { 'title' : "DHZB", 'filename': "./experiments/sinai_model_results.d" },
}


In [ ]:
def plot_cc(exp, ax, title,legend=False):    
    for mode in modes:    
        y_test = exp[mode]['y_true']
        y_probs = exp[mode]['y_probs']
        score = brier_score_loss(y_test, y_probs, pos_label=y_test.max())
        fraction_of_positives, mean_predicted_value = calibration_curve(y_test, y_probs, n_bins=10)
        ax.plot(mean_predicted_value, fraction_of_positives, label="%s (Brier=%1.3f)" % (labels[mode], score), linestyle=linestyles[mode],marker=markers[mode])    
    ax.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    ax.set_ylabel("Fraction of Positives")
    ax.set_xlabel("Mean Predicted Probability")
    ax.set_ylim([-0.05, 1.05])
    if legend: ax.legend(loc=None, fancybox=True, shadow=True)
    ax.set_title(title)

In [ ]:
''' setting up graph display structures '''
modes = ['results', 'calibrated_results']
labels = dict(zip(modes, ['No calibration','Sigmoid']))
markers = dict(zip(modes, ['s','d']))
linestyles = dict(zip(modes, ['-','--']))
algs = [algorithms.RF, algorithms.GBDT]
alg_labels = dict(zip(algs, ['RF', 'GBDT']))

fig, axs = plt.subplots(len(algs), len(cohorts), figsize=(12,8))
i = 0
''' cycle over algorithms and cohorts '''
for algorithm in algs:
    j = 0
    for cohort in cohorts:
        experiments = unpickle(cohorts[cohort]['filename'])
        exps = [exp for exp in experiments.values() if exp['parameters']['algorithm'] in [algorithm]]        
        plot_cc(exps[1], axs[i,j], f"{cohorts[cohort]['title']} ({alg_labels[algorithm]})", legend=(i==j==0))        
        j += 1
    i += 1

plt.tight_layout()
plt.savefig('cc_plots.pdf')